In [1]:
import torch

In [7]:
import pandas as pd
df = pd.read_csv('data/train.csv')
df_t = pd.read_csv('data/test.csv')

## Cleaning and storing the data

In [4]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [8]:
sex_mapper = {'male':0,'female':1}
df['Sex'] = df['Sex'].map(sex_mapper)
df_t['Sex'] = df_t['Sex'].map(sex_mapper)
        

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


In [10]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 36.0+ KB


In [11]:
def get_title(name):
    first = name.split(',')
    second = first[1].split('.')
    return second[0]

In [12]:
df.Name = df.Name.apply(get_title)
df_t.Name = df_t.Name.apply(get_title)


def best_age(name):
    return df.loc[(df.Name == name)].Age.mean()

for i in df.Name.unique():
    if (df.loc[(df.Name == i)].Age.isnull().sum()) > 0:
        print(i, "&", df.loc[(df.Name == i)].Age.isnull().sum())
        print(best_age(i))

 Mr & 119
32.368090452261306
 Mrs & 17
35.898148148148145
 Miss & 36
21.773972602739725
 Master & 4
4.574166666666667
 Dr & 1
42.0


In [14]:
df.Age=df.Age.fillna(df.Name.apply(best_age))
df_t.Age=df_t.Age.fillna(df_t.Name.apply(best_age))

In [17]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [19]:
y_train = df.Survived 
X_train = df.drop("Survived", axis=1)
X_train = X_train.drop(["Ticket", "Cabin", "PassengerId"], axis=1) # removing other 3 figures from dataset
X_train.loc[(X_train.Name == " Miss") & (X_train.SibSp == 0) & (X_train.Parch== 0) & (X_train.Embarked == "S")].count() 
X_train.loc[(X_train.Name == " Mrs") & (X_train.SibSp == 0) & (X_train.Parch== 0) & (X_train.Embarked == "S")]
X_train.Embarked.fillna("S", inplace=True)

df_t.Fare.fillna(df_t.Fare.mean(), inplace=True) # fill missing values in test dataframe

features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(X_train[features], drop_first=True) #change categorical variables into dummy variables 

test_features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X_test = pd.get_dummies(df_t[test_features], drop_first=True)
#y_test = df_t.Survived

In [22]:
X['label'] = y_train

In [40]:
import numpy as np
np.array(X,dtype=np.float32)

array([[ 3.      ,  0.      , 22.      , ...,  0.      ,  1.      ,
         0.      ],
       [ 1.      ,  1.      , 38.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 3.      ,  1.      , 26.      , ...,  0.      ,  1.      ,
         1.      ],
       ...,
       [ 3.      ,  1.      , 21.773973, ...,  0.      ,  1.      ,
         0.      ],
       [ 1.      ,  0.      , 26.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 3.      ,  0.      , 32.      , ...,  1.      ,  0.      ,
         0.      ]], dtype=float32)

In [38]:
X.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q',
       'Embarked_S', 'label'],
      dtype='object')

In [24]:
X.to_csv('data/data.csv',index=False)

In [25]:
df = pd.read_csv('data/data.csv')
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,label
0,3,0,22.0,1,0,7.2500,0,1,0
1,1,1,38.0,1,0,71.2833,0,0,1
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,1,1
4,3,0,35.0,0,0,8.0500,0,1,0


## Model

In [68]:
import torch
from torch import nn
from torch.nn import functional as F
from pathlib import Path
import pytorch_lightning as pl
#from pytorch_lightning.metrics import functional
import os


In [69]:
class TitanicDataSet(torch.utils.data.Dataset):
    def __init__(self,mode='train',data_dir='/Users/vinay/Projects/MLOPS/Pylg/data',split = [0.7,0.5]):
        self.mode = mode
        self.split = split
        if data_dir:
            self.data_dir = Path(data_dir)
        else:
            pwd = os.getcwd()
            self.data_dir = Path(pwd)/'data'
        self.data = self.get_split()
        self.input_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q',
       'Embarked_S']
        self.label_cols = ['label']
        
    def __getitem__(self,index):
        x = self.data[self.input_cols].iloc[index]
        y = self.data[self.label_cols].iloc[index]
        return np.array(x,dtype=np.float32),np.array(y,dtype=np.int32)
    
    def __len__(self):
        return len(self.data)
    
    def get_split(self):
        data = pd.read_csv(self.data_dir/'data.csv')
        
        ids = np.random.permutation(len(data))
        train_id = int(len(ids)*self.split[0])
        valid_id = train_id + int((len(ids)-train_id-1)*self.split[-1])
        if self.mode == 'train':
            return data.iloc[ids[:train_id]]
        elif self.mode == 'valid':
            return data.iloc[ids[train_id:valid_id]]
        else:
            return data.iloc[valid_id:]
        
        

In [62]:
dset = TitanicDataSet()

In [63]:
for x,y in dset:
    print(x)
    print(y)
    break

[ 3.        1.       21.773973  3.        1.       25.4667    0.
  1.      ]
[0]


In [65]:
dloader = torch.utils.data.DataLoader(dset, batch_size=10, shuffle=True)

In [66]:
for x_b,y_b in dloader:
    print(x_b.shape)
    print(y_b.shape)
    break

torch.Size([10, 8])
torch.Size([10, 1])


## Modeling

In [71]:
def accuracy(y_hat,y):
    return torch.mean(y_hat == y)



class Model(pl.LightningModule):
    def __init__(self,hparams):
        super().init()
        self.save_hyperparameters(hparams)
        self.linear1 = nn.Linear(hparams.input_sz,hparams.ll1)
        self.linear2 = nn.Linear(hparams.ll1,hparams.ll2)
        self.outlayer = nn.Linear(hparams.ll2,2)
        
    def forward(self,x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.outlayer(x)
        return x
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat,y)
        self.log('train_loss', loss, on_step=True)
        return loss
        
    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_hat = torch.argmax(y_hat, dim=1)
        acc = accuracy(y_hat, y)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        return acc
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.optimizer.lr)
        return optimizer
        
        